In [ ]:
import pandas as pd
from IPython.display import display
from io import StringIO
import ee
# pip install earthengine-api
import requests
import time
import mlflow


import src.stages.image_mining as mining_package
# import scripts.earth_engine_auth as ee_auth

n_images_class = 1000
# List of levels to keep
levels_to_merge = []

#     '52.a.05.006+52.b.08.101',
#      '52.a.05.006+51.a.01.101',
    # '52.a.05.001+52.b.09.101',
    # '61.a.02.004+55.a.02.101',
    # '61.a.04.010+55.a.02.101',
    # '75.a.02.012A+65.a.04.006+50.a.01.101+50.a.03.101+57.a.03.101',
    # '50.a.01.101+50.a.03.101',
    # '59.b.04.101+59.b.06.101',
    # '52.b.08.101+27.a.03.101',
    # '52.b.09.101',
    # '76.c.10.001+64.a.05.003+50.c.13.101',
    # '74.a.02.001+52.b.07.101+64.a.05.017+50.c.13.101+56.a.05.101'

# ee_auth.authenticate()


/Users/gabrieldiazireland/Desktop/OpenPAS/Satellites_Land_Coverage_MLOPS_training/.venv/lib/python3.11/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [ ]:
# Step 1: Run this once to authenticate
ee.Authenticate()

# Step 2: Initialize
ee.Initialize()

Exception: Problem requesting tokens. Please try again.  HTTP Error 400: Bad Request b'{\n  "error": "invalid_grant",\n  "error_description": "Malformed auth code."\n}'

### **Grouping Classes Based on Requirements**

In [2]:
df_load = mining_package.DataLoader("Sampled_Coordinates.csv")
df = df_load.read_csv()

df_customize = mining_package.PreprocessClasses(levels_to_merge,n_images_class)
df = df_customize.clean_and_filter(df)

df_final = mining_package.ImageNameProcessor()
df= df_final.process_image_names(df)

Number of ORIGINAL unique levels in the 'CVB1': 1
Number of CURRENT unique levels in the 'CVB1': 1
Unique original values in the 'CVB1': ['49.a.02.101']
Unique current values in the 'CVB1': ['49.a.02.101']


### **Visualizamos df**

In [3]:
df.head(2)

,rand_point_id,fid,COD_TESELA,SITE_CODE,COD_TV,NOMBRE,CVB1,CVB2,CVB3,CVB4,CVB5,long,lat,Image_Name,Image_Name_Length
0,0,26420,30TTK85-06-076662,ES4110002,49.a.02.101,Pastizales psicroxerófilos criorosubmediterrán...,49.a.02.101,NaN,NaN,NaN,NaN,-5.584811,40.226281,26420:49.a.02.101,17
1,1,28067,30TUK06-06-078310,ES4110002,49.a.02.101,Pastizales psicroxerófilos criorosubmediterrán...,49.a.02.101,NaN,NaN,NaN,NaN,-5.318841,40.275757,28067:49.a.02.101,17


### **Autentificación Earth Engine**

### **Función de obtención de imagenes**

##### Importante preparar las carpetas con sus nombres en google drive!

In [3]:


# help(mining_package.EE_Requests)

run_mining = mining_package.EE_Requests()

collection = 'COPERNICUS/S2_SR_HARMONIZED'
min_date = '2023-07-01'
max_date = '2023-09-01'
collection_name = 'extremadura_datos_lluvia'
square_buffer = 150

run = run_mining.Init_EE_Request(df,collection, min_date, max_date,collection_name,square_buffer)

print("TOTAL Execution time:", run['execution_time'], "min")
print("ERRORS:", run['count_error'])
print("SUCCESS:", run['count_success'])
print("ERROR LIST:", run['list_error'])
print("SUCCESS LIST:", run['list_success'])


NameError: name 'df' is not defined

In [11]:
#logging run after pipeline run 
import os
mlflow.set_tracking_uri("file:./mlruns")

# Config opcional (para evitar interferencias de entorno)
os.environ.pop("MLFLOW_TRACKING_URI", None)

mlflow.set_experiment("geospatial_experiment")

with mlflow.start_run():
    mlflow.log_param("number_images", n_images_class)
    mlflow.log_param("collection", collection)
    mlflow.log_param("min_date", min_date)
    mlflow.log_param("max_date", max_date)
    mlflow.log_param("collection_name", collection_name)
    mlflow.log_param("square_buffer", square_buffer)

    output_path = "output.txt"
    with open(output_path, "w") as f:
        f.write("Este es un archivo de salida de prueba.")

    mlflow.log_artifact(output_path)

In [ ]:
import subprocess

subprocess.Popen(["mlflow", "ui", "--backend-store-uri", "file:./mlruns"])


# open http://127.0.0.1:5000 in browser

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'file...>

/Users/gabrieldiazireland/Desktop/OpenPAS/Satellites_Land_Coverage_MLOPS_training/.venv/lib/python3.11/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251
/Users/gabrieldiazireland/Desktop/OpenPAS/Satellites_Land_Coverage_MLOPS_training/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2025-06-29 01:19:00 -0400] [16794] [INFO] Starting gunicorn 21.2.0
[2025-06-29 01:19:00 -0400] [16794] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-06-29 01:19:00 -0400] [16794] [ERROR] Retrying in 1 second.
[2025-06-29 01:19:01 -0400] [16794] [ERROR] Conn

##Nota sobre COPERNICUS HARMONIZED

COPERNICUS/S2:
The "COPERNICUS/S2" dataset in Earth Engine contains raw Sentinel-2 Level-1C (L1C) data. Level-1C data refers to the original data acquired by the satellite, which undergoes minimal processing and is provided in its original geometry and radiometric format. The imagery in this dataset includes multiple spectral bands at different spatial resolutions.

COPERNICUS/S2_SR_HARMONIZED:
The "COPERNICUS/S2_SR_HARMONIZED" dataset in Earth Engine contains harmonized Sentinel-2 Level-2A (L2A) data. Level-2A data refers to top-of-atmosphere (TOA) reflectance data that has undergone atmospheric correction and radiometric calibration. This processing step aims to remove atmospheric effects and provide a consistent and comparable reflectance product. The harmonization process takes into account various factors like atmospheric conditions, sun angle, and surface properties.